<a href="https://colab.research.google.com/github/ClaudioLucioLopes/PrescriptiveAnalysis/blob/master/3_0_Projeto_de_757_Mobility_Startup_Caso_mais_real_V01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caso da Projeto 757 Mobility Startup - Caso mais real


# Instalação e módulos necessários

In [0]:
!pip install ortools
from ortools.linear_solver import pywraplp
import numpy as np


# Organizando os dados

In [0]:


#Tempo calculado a partir da ultima atualização de tráfego e posição dos motoristas
#Geramos tempos aleatórios, mas usamos uma baseline de 30 para verificarmos a solução
Tempo_Previsto =  np.round(np.random.rand(300,200)*100+30,0)


#Número de clientes que solicitaram o serviço no bairro naquele momento
numero_clientes = Tempo_Previsto.shape[0]
numero_motoristas = Tempo_Previsto.shape[1]
print('Número de clientes: ', numero_clientes)
print('Número de motoristas: ', numero_motoristas)
print (Tempo_Previsto)


Número de clientes:  300
Número de motoristas:  200
[[119. 111.  45. ...  60.  76. 112.]
 [ 72.  78. 101. ... 126. 129. 116.]
 [ 97.  59.  61. ... 102. 126.  32.]
 ...
 [129. 123. 121. ...  32. 113. 114.]
 [127.  91.  89. ...  92. 121.  60.]
 [ 78.  48.  96. ...  52. 103. 112.]]


# Criando o modelo

In [0]:
# Criando o modelo de atendimento motoristas e clientes
modelo_atendimento = pywraplp.Solver('Modelo de atendimento',pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)


#Criando as variáveis de decisão
atendimento = {}

for i in range(numero_clientes):
    for j in range(numero_motoristas):
        atendimento[i, j] = modelo_atendimento.BoolVar('atendimento[%i,%i]' % (i, j))

#Criando a nossa função objetivo e atribuindo a minimizaçãão
modelo_atendimento.Minimize(modelo_atendimento.Sum(Tempo_Previsto[i,j] * atendimento[i,j] for i in range(numero_clientes) for j in range(numero_motoristas)))


#Restrições para os atendimentos dos clientes e dos motoristas


# Cada cliente deve ser atendido por apenas 1 motorista .
if numero_motoristas < numero_clientes:
    for i in range(numero_clientes):
        modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for j in range(numero_motoristas)]) <= 1)
else:
    for i in range(numero_clientes):
        modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for j in range(numero_motoristas)]) == 1)
    

# Se o número de clientes for maior que o número de motoristas vamos garantir que os atendimentos mais rápidos aconteçam.
if numero_motoristas < numero_clientes:
    modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for i in range(numero_clientes) for j in range(numero_motoristas)]) >= numero_motoristas)

# Cada motorista deve atender no máximo 1 pessoa naquele instante de tempo .
for j in range(numero_motoristas):
    modelo_atendimento.Add(modelo_atendimento.Sum([atendimento[i, j] for i in range(numero_clientes)]) <= 1)





# Executando o modelo

In [0]:
#Executando o modelo


print('Número de restrições: ',modelo_atendimento.NumConstraints())
print('Número de variáveis: ',modelo_atendimento.NumVariables())
modelo_atendimento.EnableOutput()
modelo_atendimento.Solve()

print('Tempo: ', modelo_atendimento.WallTime(), 'ms')
print('Iterações: ', modelo_atendimento.iterations())

Número de restrições:  501
Número de variáveis:  60000
Tempo:  3406 ms
Iterações:  0


# Avaliando os resultados

In [0]:
print('Tempo total para os atendimentos: ', round(modelo_atendimento.Objective().Value(),2))

print()
for i in range(numero_clientes):
    for j in range(numero_motoristas):
        #print(atendimento[i, j].solution_value() )
        if atendimento[i, j].solution_value() > 0:
            print('Cliente %d será atendido pelo motorista %d.  Tempo previsto = %d' % (i+1,j+1,Tempo_Previsto[i,j]))
print()



Tempo total para os atendimentos:  6061.0

Cliente 2 será atendido pelo motorista 137.  Tempo previsto = 30
Cliente 3 será atendido pelo motorista 37.  Tempo previsto = 31
Cliente 5 será atendido pelo motorista 134.  Tempo previsto = 30
Cliente 6 será atendido pelo motorista 88.  Tempo previsto = 31
Cliente 7 será atendido pelo motorista 132.  Tempo previsto = 31
Cliente 8 será atendido pelo motorista 145.  Tempo previsto = 30
Cliente 10 será atendido pelo motorista 167.  Tempo previsto = 30
Cliente 11 será atendido pelo motorista 23.  Tempo previsto = 30
Cliente 12 será atendido pelo motorista 22.  Tempo previsto = 31
Cliente 15 será atendido pelo motorista 74.  Tempo previsto = 30
Cliente 16 será atendido pelo motorista 124.  Tempo previsto = 30
Cliente 17 será atendido pelo motorista 156.  Tempo previsto = 30
Cliente 18 será atendido pelo motorista 122.  Tempo previsto = 30
Cliente 19 será atendido pelo motorista 45.  Tempo previsto = 30
Cliente 21 será atendido pelo motorista 7.  T